<a href="https://colab.research.google.com/github/thehapyone/Thesis_Project/blob/master/Entity_Embeddings_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learned Embeddings
This program will attempt to create embeddings for the categorical features.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
# using the latest updated file
#data = pd.read_csv("drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v3_feature_engineering_data.csv", sep=',', low_memory=False)
# data = pd.read_csv("drive/My Drive/Thesis/data/Data_Engineering/Edited_Raw_Data/v2_Sorted_Database.csv", sep=',', low_memory=False)

In [38]:
data.shape

(336677, 447)

In [0]:
# the BIWEEK_SEND_DATE is a datetime feature and not just normal object date
data['BIWEEK_SEND_DATE'] = pd.to_datetime(data['BIWEEK_SEND_DATE'])

# Categorical Features
We have 70 categorical features inlcuding the Chassis feature.

The quesion pondering my mind is? Should the chassis feature be discared or used?

In [40]:
# find all categorical features.
data_cat = data.select_dtypes(include='object')
data_cat

,T_CHASSIS,DELIVERY_DATE,LAST_RUN,DELIVERY_DATE.1,COUNTRY_OF_OPERATION,VAR_7MA_FRONT_AXLE_TYPE,VAR_DPX_ENGINE_TYPE,VAR_T1X_PTO_ENGINE_MOUNTED_REAR,VAR_4DX_FRONT_BUMPER,VAR_F1X_REAR_AXLE_LOAD,VAR_FTX_GROUND_CLEARANCE_FA,VAR_CKX_VEHICLE_PROFILE,VAR_KTX_ENGINE_EMISSION_CONTROL,VAR_3ZB_TRANSMISSION_PTO_TORQUE_CAP,VAR_QCX_TOPOGRAPHY,VAR_YLX_REAR_SUSPENSION_SYSTEM,VAR_DKX_GROSS_COMBINATION_WEIGHT,VAR_2RB_BATTERY_MAINTENANCE,VAR_FAX_STEERING_WHEEL_POS,VAR_2CX_CAB_VERSION,VAR_2DX_REAR_CAB_SUSPENSION,VAR_GSX_CHASSIS_HEIGHT,VAR_FFX_PUSHER_AXLE_FEATURES,VAR_5CB_AIR_SUSP_FEATURE_SOFTWARE,VAR_2GB_CLUTCHABLE_AIR_COMPRESSOR,VAR_5NX_ENGINE_BLOCK_HEATER,VAR_TAX_REAR_AXLE_RATIO,VAR_A3A_IDLE_SHUTDOWN,VAR_2LX_FRONT_CAB_SUSPENSION,VAR_RUX_GEAR_SHIFTING_SYSTEM,VAR_U3X_BRAKE_FRONT,VAR_T4X_PTO_TRANSM_MOUNTED,VAR_UBX_HYDRAULIC_PUMP_ENG_MTD,VAR_4QB_BODYBUILDER_EL_INTERFCAB_CHAS,VAR_REX_CRANK_CASE_VENTILATION,VAR_78X_TRANSPORT_CYCLE,VAR_T2X_TRANSMISSIONRETARDER_COOLER,VAR_U4X_BRAKE_DRIVE_REAR,VAR_FVX_FRONT_AXLE_LOAD,VAR_VWX_COMPRESSOR_CAPACITY,VAR_DAX_AXLE_ARRANGEMENT,VAR_H1B_TURBO_APPLICATION,VAR_FDX_REAR_AXLE_ARRANGEMENT,VAR_7PX_HYDRAULIC_PUMP_GEARBOX_MOUNTED,VAR_7NA_FRONT_SUSPENSION_INSTALLATION,VAR_LKX_DRIVER_STATE_SENSING_SYSTEM,VAR_QWX_TURBO,VAR_FZX_FRONT_AXLE_ARRANGEMENT,VAR_V6X_RETARDER,VAR_K4C_VEHICLE_UPTIME_SETTINGS,VAR_0RA_REGENERATION_CONTROLS_DPF,VAR_YKX_FRONT_SUSPENSION_SYSTEM,VAR_T3X_PTO_ENGINE_REAR_MOUNTED,VAR_PJX_ALTERNATOR_CAPACITY,VAR_41X_PARKING_HEATER,VAR_D7C_WHOLE_VEHICLE_TYPE_APPROVAL,VAR_5JB_TELEMATICS_COMMUNICAT_GATEWAYS,VAR_RWX_TRANSMISSION,VAR_DHX_ROAD_CONDITION,VAR_2EX_CAB_TILT_PUMP,VAR_D9C_EMISSION_LEVEL,VAR_DDX_CHASSIS_TYPE,VAR_O8A_ENGINE_PTO_TORQUE_CAPACITY,VAR_33A_CAB_HEIGHT,VAR_J2A_ACTIVE_STEERING,VAR_N9C_ADR_CLASSIFICATION,VFE_0001_HORSE_POWER,VFE_0002_GROSS_COMBINATION_WEIGHT_BIN,VFE_0003_ENGINE_GENERATION,VFE_0004_REAR_AXLE_LOAD_BIN,VFE_0005_REAR_AXLE_RATIO_BIN,VFE_0009_LTVF_TYPE,VFE_0006_VEHICLE_OPERATION_DIGIT2,VFE_0007_CAB_HEIGHT,LAST_RUN.1
0,O-774932,2016-06-02,2019-05-22 01:50:17,2016-06-02,DEU,FAT-RIG,ENG-VE13,UPTER,BUMP-B,RAL13,FA-MED,TYPE-FH,EM-EU6,TPTT600,T-PFLAT,RSS-AIR,GCW44.0,UBAMAFRE,STWPOS-L,CAB-HSLP,RCABS-M,CHH-MED,UPAXLE,ASF-DL3,UCLUAIRC,UEBHEAT,RAT2.64,UEIDLESD,FCABS-M,GSS-AGS,BF-DILW,WithPTTR,UHPE,UBBCHAS,CCV-C,TC-LONGD,UTCOOL,BR-DILW,FAL7.5,2COMP900,4*2,TURBO-S,RAA11,UHPG,FSI-LEAF,UDAS,TURB-HD,FAA1,URETARD,VUPT-A1,DRC-AUTO,FSS-LEAF,UPTOENGR,24AL150B,PH-CAB2,WVTA-EC,TGW-3GWL,AT261,RC-SMOOT,CTILTP-M,EML-BAS,TRACTOR,UEPTT,CABH215,UACTST,ADRC-FL,Low,Med,K,RAL13,RAL13,INTERREG_HAUL,8,CABH215,2019-05-14 10:32:00
1,O-774932,2016-06-02,2019-05-22 01:50:17,2016-06-02,DEU,FAT-RIG,ENG-VE13,UPTER,BUMP-B,RAL13,FA-MED,TYPE-FH,EM-EU6,TPTT600,T-PFLAT,RSS-AIR,GCW44.0,UBAMAFRE,STWPOS-L,CAB-HSLP,RCABS-M,CHH-MED,UPAXLE,ASF-DL3,UCLUAIRC,UEBHEAT,RAT2.64,UEIDLESD,FCABS-M,GSS-AGS,BF-DILW,WithPTTR,UHPE,UBBCHAS,CCV-C,TC-LONGD,UTCOOL,BR-DILW,FAL7.5,2COMP900,4*2,TURBO-S,RAA11,UHPG,FSI-LEAF,UDAS,TURB-HD,FAA1,URETARD,VUPT-A1,DRC-AUTO,FSS-LEAF,UPTOENGR,24AL150B,PH-CAB2,WVTA-EC,TGW-3GWL,AT261,RC-SMOOT,CTILTP-M,EML-BAS,TRACTOR,UEPTT,CABH215,UACTST,ADRC-FL,Low,Med,K,RAL13,RAL13,INTERREG_HAUL,8,CABH215,2019-05-14 10:32:00
2,O-774932,2016-06-02,2019-05-22 01:50:17,2016-06-02,DEU,FAT-RIG,ENG-VE13,UPTER,BUMP-B,RAL13,FA-MED,TYPE-FH,EM-EU6,TPTT600,T-PFLAT,RSS-AIR,GCW44.0,UBAMAFRE,STWPOS-L,CAB-HSLP,RCABS-M,CHH-MED,UPAXLE,ASF-DL3,UCLUAIRC,UEBHEAT,RAT2.64,UEIDLESD,FCABS-M,GSS-AGS,BF-DILW,WithPTTR,UHPE,UBBCHAS,CCV-C,TC-LONGD,UTCOOL,BR-DILW,FAL7.5,2COMP900,4*2,TURBO-S,RAA11,UHPG,FSI-LEAF,UDAS,TURB-HD,FAA1,URETARD,VUPT-A1,DRC-AUTO,FSS-LEAF,UPTOENGR,24AL150B,PH-CAB2,WVTA-EC,TGW-3GWL,AT261,RC-SMOOT,CTILTP-M,EML-BAS,TRACTOR,UEPTT,CABH215,UACTST,ADRC-FL,Low,Med,K,RAL13,RAL13,INTERREG_HAUL,8,CABH215,2019-05-14 10:32:00
3,O-774932,2016-06-02,2019-05-22 01:50:17,2016-06-02,DEU,FAT-RIG,ENG-VE13,UPTER,BUMP-B,RAL13,FA-MED,TYPE-FH,EM-EU6,TPTT600,T-PFLAT,RSS-AIR,GCW44.0,UBAMAFRE,STWPOS-L,CAB-HSLP,RCABS-M,CHH-MED,UPAXLE,ASF-DL3,UCLUAIRC,UEBHEAT,RAT2.64,UEID

In [41]:
# it seems some categorical features have the same values i.e std of 0
data_cat['VAR_2RB_BATTERY_MAINTENANCE'].unique()

array(['UBAMAFRE', 'BAMAFREE'], dtype=object)

In [34]:
## find categorical features with zero variance - Will don't need those features.
data_cat.columns
var_1_categories = []
for col in data_cat.columns:
  unique_Values = data_cat.loc[:, col].nunique()
  print (col, unique_Values)
  if unique_Values == 1:
    var_1_categories.append(col)


T_CHASSIS 445
VAR_DAX_AXLE_ARRANGEMENT 4
COUNTRY_OF_OPERATION 1
VAR_7MA_FRONT_AXLE_TYPE 2
VAR_DPX_ENGINE_TYPE 3
VAR_T1X_PTO_ENGINE_MOUNTED_REAR 2
VAR_4DX_FRONT_BUMPER 3
VAR_F1X_REAR_AXLE_LOAD 9
VAR_FTX_GROUND_CLEARANCE_FA 3
VAR_CKX_VEHICLE_PROFILE 2
VAR_KTX_ENGINE_EMISSION_CONTROL 2
VAR_3ZB_TRANSMISSION_PTO_TORQUE_CAP 5
VAR_QCX_TOPOGRAPHY 3
VAR_YLX_REAR_SUSPENSION_SYSTEM 2
VAR_DKX_GROSS_COMBINATION_WEIGHT 9
VAR_2RB_BATTERY_MAINTENANCE 2
VAR_FAX_STEERING_WHEEL_POS 1
VAR_2CX_CAB_VERSION 5
VAR_2DX_REAR_CAB_SUSPENSION 2
VAR_GSX_CHASSIS_HEIGHT 4
VAR_FFX_PUSHER_AXLE_FEATURES 2
VAR_5CB_AIR_SUSP_FEATURE_SOFTWARE 3
VAR_2GB_CLUTCHABLE_AIR_COMPRESSOR 2
VAR_5NX_ENGINE_BLOCK_HEATER 2
VAR_TAX_REAR_AXLE_RATIO 22
VAR_A3A_IDLE_SHUTDOWN 2
VAR_2LX_FRONT_CAB_SUSPENSION 2
VAR_RUX_GEAR_SHIFTING_SYSTEM 3
VAR_U3X_BRAKE_FRONT 3
VAR_T4X_PTO_TRANSM_MOUNTED 2
VAR_UBX_HYDRAULIC_PUMP_ENG_MTD 2
VAR_4QB_BODYBUILDER_EL_INTERFCAB_CHAS 3
VAR_REX_CRANK_CASE_VENTILATION 3
VAR_78X_TRANSPORT_CYCLE 3
VAR_T2X_TRANSMISSIONRETA

In [35]:
# Categorical Variables with only 1 value. i.e std is 0
var_1_categories

['COUNTRY_OF_OPERATION', 'VAR_FAX_STEERING_WHEEL_POS']